#### torch.utils.data里面有很多数据预处理的包
* Dataset是一个抽象类，其他的数据集要继承他，并且重写它里面的\__getitem__(给定索引的情况下获取数据和标签)和\__len__(提供数据集大小)方法
* DataLoader batch来批量读取数据，且可以并行加速获取数据
    * num_workers就是要用多少个进程并行加载数据
    * collate_fn如何将多个样本组成一个batch
    * pin_memory 是否将数据集加载到pin memory区，从pin_memory转到GPU上上面快一些
    * drop_last 因为划分batch的时候一般最后一个batch都装不满，就看你要不要丢掉它
    * DataLoader它不是迭代器，需要先iter一下再next才行
* randomsplit 把数据集拆分成非重叠的给定长度的新数据集
#### torchvision是视觉处理工具，独立于Pytorch的，它包括
* datasets 官方提前封装好的数据集下载
* models 图像处理方面各种典型的网络，如Resnet,Alexnet，Inception等，以及一些预训练模型
* transforms 常用的数据预处理操作
* utils(utility-实体的意思) 包含make_grid(将多张图片拼接在一个网格中)和save_imag，它能将Tensor保存成图片

#### 重点讲一下transforms和datasets里面的ImageFolder

#### Crop是裁剪的意思
#### ImageFolder的参数是路径，路径下的每个文件名字都是标签，每个文件名下的内容都是对应于同样的标签的，所以可以直接用ImageFolder来构造dataset的
#### tensorBoardX是可以可视化神经网络和其训练过程的只需要pip install 一下就可以了,comment参数是作为文件名的后缀的

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from tensorboardX import SummaryWriter

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20)

    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(x) + F.relu(-x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.bn(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [8]:
#定义输入
input = torch.rand(32, 1, 28, 28)
#实例化神经网络
model = Net()
#将model保存为graph
with SummaryWriter(log_dir=r'D:\juypter_files\pytorch\logs',comment='Net') as w:
    w.add_graph(model, (input, ))


* 切换到logs目录下面，然后运行这个就行:tensorboard --logdir=./